In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

     |████████████████████████████████| 274 kB 7.0 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=9e3d9796d26e9ff871f84ec647831df45ab7ef67fa73475dbe2c48456db64089
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 4.3 MB/s 
     |████████████████████████████████| 130 kB 8.5 MB/s 
     |████████████████████████████████| 74 kB 4.6 MB/s 
     |████████████████████████████████| 145 kB 75.8 MB/s 
     |████████████████████████████████| 49 kB 7.9 MB/s 
     |████████████████████████████████| 843 kB 73.2 MB/s 
     |████████████████████████████████

In [1]:
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git


In [1]:
datapath = '/content/drive/MyDrive/PyPSA_Africa_images/datasets'

import sys
sys.argv = ['']

In [2]:
# Add pypsa africa tool
import os
os.chdir('/content/drive/MyDrive/PyPSA_Africa_images/unbiased-teacher/')

import sys
sys.path.append('/content/drive/MyDrive/PyPSA_Africa_images/detect_energy/src/')

In [3]:
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import default_argument_parser, default_setup, launch

from ubteacher import add_ubteacher_config
from ubteacher.engine.trainer import UBTeacherTrainer, BaselineTrainer

# hacky way to register
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN
from ubteacher.modeling.proposal_generator.rpn import PseudoLabRPN
from ubteacher.modeling.roi_heads.roi_heads import StandardROIHeadsPseudoLab
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN
import ubteacher.data.datasets.builtin

from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel

# import some of our own methods
from utils.detectron_utils import eval_predictor
from utils.image_utils import get_true_images

In [4]:
def main(args):
    cfg = setup(args)

    if cfg.SEMISUPNET.Trainer == "ubteacher":
        Trainer = UBTeacherTrainer
    elif cfg.SEMISUPNET.Trainer == "baseline":
        Trainer = BaselineTrainer
    else:
        raise ValueError("Trainer Name is not found.")

    trainer = Trainer(cfg)    
    trainer.train()
    
    return None

In [5]:
from detectron2 import model_zoo

def setup(args):

    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    # print(cfg)
    add_ubteacher_config(cfg)
    # print(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    # cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))
    cfg.freeze()
    default_setup(cfg, args)
    return cfg


In [6]:
import os
import time
import logging
import torch
from torch.nn.parallel import DistributedDataParallel
from fvcore.nn.precise_bn import get_bn_modules
import numpy as np
from collections import OrderedDict

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.engine import DefaultTrainer, SimpleTrainer, TrainerBase
from detectron2.engine.train_loop import AMPTrainer
from detectron2.utils.events import EventStorage
from detectron2.evaluation import COCOEvaluator, verify_results, PascalVOCDetectionEvaluator, DatasetEvaluators
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.engine import hooks
from detectron2.structures.boxes import Boxes
from detectron2.structures.instances import Instances
from detectron2.utils.env import TORCH_VERSION
from detectron2.data import MetadataCatalog
from detectron2.modeling import build_model
from detectron2 import model_zoo

from ubteacher.data.build import (
    build_detection_semisup_train_loader,
    build_detection_test_loader,
    build_detection_semisup_train_loader_two_crops,
)
from ubteacher.data.dataset_mapper import DatasetMapperTwoCropSeparate
from ubteacher.engine.hooks import LossEvalHook
from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel
from ubteacher.checkpoint.detection_checkpoint import DetectionTSCheckpointer
from ubteacher.solver.build import build_lr_scheduler

In [7]:
import numpy as np
import logging
import operator

from detectron2.modeling import build_model
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.data.build import get_detection_dataset_dicts, build_batch_data_loader
from detectron2.data.common import DatasetFromList, MapDataset
from detectron2.data.samplers import TrainingSampler
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.utils.comm import get_world_size
from detectron2.data.build import (
    trivial_batch_collator,
    worker_init_reset_seed,
    get_detection_dataset_dicts,
    build_batch_data_loader,
)


from ubteacher.data.build import divide_label_unlabel
from ubteacher.data.common import (
    AspectRatioGroupedSemiSupDatasetTwoCrop,
)
from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel

from itertools import product


# batch data loader
def build_semisup_batch_data_loader_two_crop(
    dataset,
    sampler,
    total_batch_size_label,
    total_batch_size_unlabel,
    *,
    aspect_ratio_grouping=False,
    num_workers=0
):
    world_size = get_world_size()
    assert (
        total_batch_size_label > 0 and total_batch_size_label % world_size == 0
    ), "Total label batch size ({}) must be divisible by the number of gpus ({}).".format(
        total_batch_size_label, world_size
    )

    assert (
        total_batch_size_unlabel > 0 and total_batch_size_unlabel % world_size == 0
    ), "Total unlabel batch size ({}) must be divisible by the number of gpus ({}).".format(
        total_batch_size_label, world_size
    )

    batch_size_label = total_batch_size_label // world_size
    batch_size_unlabel = total_batch_size_unlabel // world_size

    label_dataset, unlabel_dataset = dataset
    label_sampler, unlabel_sampler = sampler

    if aspect_ratio_grouping:
        label_data_loader = torch.utils.data.DataLoader(
            label_dataset,
            sampler=label_sampler,
            num_workers=num_workers,
            batch_sampler=None,
            collate_fn=operator.itemgetter(
                0
            ),  # don't batch, but yield individual elements
            worker_init_fn=worker_init_reset_seed,
        )  # yield individual mapped dict
        unlabel_data_loader = torch.utils.data.DataLoader(
            unlabel_dataset,
            sampler=unlabel_sampler,
            num_workers=num_workers,
            batch_sampler=None,
            collate_fn=operator.itemgetter(
                0
            ),  # don't batch, but yield individual elements
            worker_init_fn=worker_init_reset_seed,
        )  # yield individual mapped dict
        return AspectRatioGroupedSemiSupDatasetTwoCrop(
            (label_data_loader, unlabel_data_loader),
            (batch_size_label, batch_size_unlabel),
        )
    else:
        raise NotImplementedError("ASPECT_RATIO_GROUPING = False is not supported yet")


def build_maxar_loader_semisup_two_crops(cfg, mapper=None):
    '''
    Sets up loader for custom maxar data
    '''

    # register used datasets
    ds_names = ['fake_maxar', 'maxar']
    modes = ['train', 'val']

    for name, mode in product(ds_names, modes):

        ds_name = f'{name}_{mode}'
        json_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/{ds_name}/labels.json'
        ds_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/{ds_name}/data/' 
    
        if ds_name in DatasetCatalog.list():
            DatasetCatalog.remove(ds_name)
            MetadataCatalog.remove(ds_name)
    
        register_coco_instances(ds_name, {}, json_path, ds_path)
    
    unlabel_dicts = get_detection_dataset_dicts(
                            ['maxar_train', 'maxar_val'],
                            filter_empty=False,
                            min_keypoints=0,
                            proposal_files=None
                        )

    label_dicts = get_detection_dataset_dicts(
                            ['fake_maxar_train', 'fake_maxar_val'],
                            filter_empty=False,
                            min_keypoints=0,
                            proposal_files=None
                        )
 
    # cfg.DATASETS.SUP_PERCENT = int(100 * len(label_dicts) / (len(label_dicts) + len(unlabel_dicts)))
     
    label_dataset = DatasetFromList(label_dicts, copy=False)
    unlabel_dataset = DatasetFromList(unlabel_dicts, copy=False)

    if mapper is None:
        mapper = DatasetMapper(cfg, True)
    label_dataset = MapDataset(label_dataset, mapper)
    unlabel_dataset = MapDataset(unlabel_dataset, mapper)

    sampler_name = cfg.DATALOADER.SAMPLER_TRAIN
    logger = logging.getLogger(__name__)
    logger.info("Using training sampler {}".format(sampler_name))
    if sampler_name == "TrainingSampler":
        label_sampler = TrainingSampler(len(label_dataset))
        unlabel_sampler = TrainingSampler(len(unlabel_dataset))
    elif sampler_name == "RepeatFactorTrainingSampler":
        raise NotImplementedError("{} not yet supported.".format(sampler_name))
    else:
        raise ValueError("Unknown training sampler: {}".format(sampler_name))
    return build_semisup_batch_data_loader_two_crop(
        (label_dataset, unlabel_dataset),
        (label_sampler, unlabel_sampler),
        cfg.SOLVER.IMG_PER_BATCH_LABEL,
        cfg.SOLVER.IMG_PER_BATCH_UNLABEL,
        aspect_ratio_grouping=cfg.DATALOADER.ASPECT_RATIO_GROUPING,
        num_workers=cfg.DATALOADER.NUM_WORKERS,
    )

# cfg.DATASETS.TRAIN = ['maxar_train', 'maxar_val']




In [8]:
import logging
import torchvision.transforms as transforms
from ubteacher.data.transforms.augmentation_impl import GaussianBlur

def build_strong_maxar_augmentation(cfg, is_train):

    logger = logging.getLogger(__name__)
    augmentation = []

    if is_train:
        # This is simialr to SimCLR https://arxiv.org/abs/2002.05709
        augmentation.append(
            transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8)
        )
        augmentation.append(transforms.RandomGrayscale(p=0.2))
        augmentation.append(transforms.RandomApply([GaussianBlur([0.1, 2.0])], p=0.5))

        other_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                # transforms.RandomErasing(
                #     p=0.7, scale=(0.05, 0.2), ratio=(0.3, 3.3), value="random"
                # ),
                # transforms.RandomErasing(
                #     p=0.5, scale=(0.02, 0.2), ratio=(0.1, 6), value="random"
                # ),
                # transforms.RandomErasing(
                #     p=0.3, scale=(0.02, 0.2), ratio=(0.05, 8), value="random"
                # ),
                transforms.ToPILImage(),
            ]
        )
        augmentation.append(other_transform)

        logger.info("Augmentations used in training: " + str(augmentation))
    return transforms.Compose(augmentation)


In [9]:
from detectron2.data.dataset_mapper import DatasetMapper
import detectron2.data.detection_utils as utils

from ubteacher.data.dataset_mapper import DatasetMapperTwoCropSeparate


class MaxarDatasetMapper(DatasetMapperTwoCropSeparate):
    def __init__(self, cfg, is_train=True):
        self.augmentation = utils.build_augmentation(cfg, is_train)
        self.compute_tight_boxes = False
        self.strong_augmentation = build_strong_maxar_augmentation(cfg, is_train)

        # fmt: off
        self.img_format = cfg.INPUT.FORMAT
        self.mask_on = cfg.MODEL.MASK_ON
        self.mask_format = cfg.INPUT.MASK_FORMAT
        self.keypoint_on = cfg.MODEL.KEYPOINT_ON
        self.load_proposals = cfg.MODEL.LOAD_PROPOSALS
        # fmt: on
        if self.keypoint_on and is_train:
            self.keypoint_hflip_indices = utils.create_keypoint_hflip_indices(
                cfg.DATASETS.TRAIN
            )
        else:
            self.keypoint_hflip_indices = None

        if self.load_proposals:
            self.proposal_min_box_size = cfg.MODEL.PROPOSAL_GENERATOR.MIN_SIZE
            self.proposal_topk = (
                cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN
                if is_train
                else cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TEST
            )
        self.is_train = is_train



In [9]:
from detectron2.modeling.meta_arch.build import META_ARCH_REGISTRY
from detectron2.modeling.meta_arch.rcnn import GeneralizedRCNN

if 'MaxarPseudoLabGeneralizedRCNN' in META_ARCH_REGISTRY:
    META_ARCH_REGISTRY.__dict__['_obj_map'].pop('MaxarPseudoLabGeneralizedRCNN')

@META_ARCH_REGISTRY.register()
class MaxarPseudoLabGeneralizedRCNN(GeneralizedRCNN):

    def forward(
        self, batched_inputs, branch="supervised", given_proposals=None, val_mode=False
    ):
        if (not self.training) and (not val_mode) and (not branch=='unsup_data_weak'):
            print('actually we are doing this')
            return self.inference(batched_inputs)

        images = self.preprocess_image(batched_inputs)

        if "instances" in batched_inputs[0]:
            gt_instances = [x["instances"].to(self.device) for x in batched_inputs]
        else:
            gt_instances = None

        features = self.backbone(images.tensor)

        if branch == "supervised":
            # Region proposal network
            proposals_rpn, proposal_losses = self.proposal_generator(
                images, features, gt_instances
            )

            # # roi_head lower branch
            _, detector_losses = self.roi_heads(
                images, features, proposals_rpn, gt_instances, branch=branch
            )

            losses = {}
            losses.update(detector_losses)
            losses.update(proposal_losses)
            return losses, [], [], None

        elif branch == "unsup_data_weak":

            # Region proposal network
            proposals_rpn, _ = self.proposal_generator(
                images, features, None
            )

            # roi_head lower branch (keep this for further production)  
            # notice that we do not use any target in ROI head to do inference !

            # if self.roi_heads.training:
            #    kwargs = {'targets': None, 'compute_loss': False, 'branch': branch} 
            # else:
            #    kwargs = {'targets': None, 'branch': branch} 

            proposals_roih, ROI_predictions = self.roi_heads(
                images,
                features,
                proposals_rpn,
                targets=None,
                compute_loss=False,
                branch=branch,
            )

            print('Here still all is good?')
            return {}, proposals_rpn, proposals_roih, ROI_predictions

        elif branch == "val_loss":
 
            # Region proposal network
            proposals_rpn, proposal_losses = self.proposal_generator(
                images, features, gt_instances, compute_val_loss=True
            )

            # roi_head lower branch
            _, detector_losses = self.roi_heads(
                images,
                features,
                proposals_rpn,
                gt_instances,
                branch=branch,
                compute_val_loss=True,
            )

            losses = {}
            losses.update(detector_losses)
            losses.update(proposal_losses)
            return losses, [], [], None

In [ ]:
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN

def change_classes(data):
    for i, datum in enumerate(data):
        print(datum['instances'])
        classes = getattr(datum['instances'], 'gt_classes')
        setattr(datum['instances'], 'gt_classes', torch.ones_like(classes, dtype=int))
        data[i] = datum
    
    return data


class UBTeacherTrainer(DefaultTrainer):
    def __init__(self, cfg):
        """
        Args:
            cfg (CfgNode):
        Use the custom checkpointer, which loads other backbone models
        with matching heuristics.
        """
        cfg = DefaultTrainer.auto_scale_workers(cfg, comm.get_world_size())

        # frcnn = 'faster_rcnn_R_101_FPN_3x.yaml'
        # frcnn_cfg = get_cfg()
        # frcnn_cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/"+frcnn))
        # frcnn_cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/PyPSA_Africa_images/PISA_models/11_01_2022_fake_maxar_train/model_final.pth'
        # frcnn_cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

        # frcnn_model = build_model(frcnn_cfg)

        # create an student model
        # model = MaxarPseudoLabGeneralizedRCNN(cfg).to(torch.device('cuda'))

        # print('before we set it to our model')
        # putting the trained network into the unsup model
        # model.proposal_generator = frcnn_model.proposal_generator
        # model.backbone = frcnn_model.backbone
        # model.roi_heads = model.roi_heads

        model = self.build_model(cfg)
        data_loader = self.build_train_loader(cfg)
        optimizer = self.build_optimizer(cfg, model)

        # create an teacher model
        # model_teacher = MaxarPseudoLabGeneralizedRCNN(cfg).to(torch.device('cuda'))
        model_teacher = self.build_model(cfg)
        print('built model')
        print(type(model_teacher))
        
        # cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/"+frcnn))
        
        # model_teacher.proposal_generator = frcnn_model.proposal_generator
        # model_teacher.backbone = frcnn_model.backbone
        # model_teacher.roi_heads = model.roi_heads
        
        # print('Hot fixing')
        #cfg.defrost()
        # cfg.MODEL.RESNETS.OUT_FEATURES = ['res5']
        # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["res5"]
        # cfg.freeze()

        
        self.model_teacher = model_teacher
        # For training, wrap with DDP. But don't need this for inference.
        if comm.get_world_size() > 1:
            model = DistributedDataParallel(
                model, device_ids=[comm.get_local_rank()], broadcast_buffers=False
            )

        TrainerBase.__init__(self)
        self._trainer = (AMPTrainer if cfg.SOLVER.AMP.ENABLED else SimpleTrainer)(
            model, data_loader, optimizer
        )
        self.scheduler = self.build_lr_scheduler(cfg, optimizer)

        # Ensemble teacher and student model is for model saving and loading
        ensem_ts_model = EnsembleTSModel(model_teacher, model)

        self.checkpointer = DetectionTSCheckpointer(
            ensem_ts_model,
            cfg.OUTPUT_DIR,
            optimizer=optimizer,
            scheduler=self.scheduler,
        )
        self.start_iter = 0
        self.max_iter = cfg.SOLVER.MAX_ITER
        self.cfg = cfg

        self.register_hooks(self.build_hooks())
        print('Initialized ub teacher')

        return


    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapperTwoCropSeparate(cfg, True)
        return build_maxar_loader_semisup_two_crops(cfg, mapper)

    def train(self):
        self.train_loop(self.start_iter, self.max_iter)
        if hasattr(self, "_last_eval_results") and comm.is_main_process():
            verify_results(self.cfg, self._last_eval_results)
            return self._last_eval_results


    def train_loop(self, start_iter : int, max_iter : int):
        logger = logging.getLogger(__name__)
        logger.info("Starting training from iteration {}".format(start_iter))

        self.iter = self.start_iter = start_iter
        self.max_iter = max_iter

        with EventStorage(start_iter) as self.storage:
            try:
                self.before_train()

                for self.iter in range(start_iter, max_iter):
                    self.before_step()
                    self.run_step_full_semisup()
                    break
                    self.after_step()
            except Exception:
                logger.exception("Exception during training:")
                raise
            finally:
                self.after_train()

    # =====================================================
    # ================== Pseduo-labeling ==================
    # =====================================================
    def threshold_bbox(self, proposal_bbox_inst, thres=0.7, proposal_type="roih"):
        if proposal_type == "rpn":
            valid_map = proposal_bbox_inst.objectness_logits > thres

            # create instances containing boxes and gt_classes
            image_shape = proposal_bbox_inst.image_size
            new_proposal_inst = Instances(image_shape)

            # create box
            new_bbox_loc = proposal_bbox_inst.proposal_boxes.tensor[valid_map, :]
            new_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            new_proposal_inst.gt_boxes = new_boxes
            new_proposal_inst.objectness_logits = proposal_bbox_inst.objectness_logits[
                valid_map
            ]
        elif proposal_type == "roih":
            valid_map = proposal_bbox_inst.scores > thres

            # create instances containing boxes and gt_classes
            image_shape = proposal_bbox_inst.image_size
            new_proposal_inst = Instances(image_shape)

            # create box
            new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor[valid_map, :]
            new_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            new_proposal_inst.gt_boxes = new_boxes
            new_proposal_inst.gt_classes = proposal_bbox_inst.pred_classes[valid_map]
            new_proposal_inst.scores = proposal_bbox_inst.scores[valid_map]

        return new_proposal_inst

    def process_pseudo_label(
        self, proposals_rpn_unsup_k, cur_threshold, proposal_type, psedo_label_method=""
    ):
        list_instances = []
        num_proposal_output = 0.0
        for proposal_bbox_inst in proposals_rpn_unsup_k:
            # thresholding
            if psedo_label_method == "thresholding":
                proposal_bbox_inst = self.threshold_bbox(
                    proposal_bbox_inst, thres=cur_threshold, proposal_type=proposal_type
                )
            else:
                raise ValueError("Unkown pseudo label boxes methods")
            num_proposal_output += len(proposal_bbox_inst)
            list_instances.append(proposal_bbox_inst)
        num_proposal_output = num_proposal_output / len(proposals_rpn_unsup_k)
        return list_instances, num_proposal_output

    def remove_label(self, label_data):
        for label_datum in label_data:
            if "instances" in label_datum.keys():
                del label_datum["instances"]
        return label_data

    def add_label(self, unlabled_data, label):
        for unlabel_datum, lab_inst in zip(unlabled_data, label):
            unlabel_datum["instances"] = lab_inst
        return unlabled_data

    # =====================================================
    # =================== Training Flow ===================
    # =====================================================

    def run_step_full_semisup(self):
        self._trainer.iter = self.iter
        assert self.model.training, "[UBTeacherTrainer] model was changed to eval mode!"
        start = time.perf_counter()
        data = next(self._trainer._data_loader_iter)
        # data_q and data_k from different augmentations (q:strong, k:weak)
        # label_strong, label_weak, unlabed_strong, unlabled_weak
        label_data_q, label_data_k, unlabel_data_q, unlabel_data_k = data
        data_time = time.perf_counter() - start

        # remove unlabeled data labels
        unlabel_data_q = self.remove_label(unlabel_data_q)
        unlabel_data_k = self.remove_label(unlabel_data_k)

        # burn-in stage (supervised training with labeled data)
        # if self.iter < self.cfg.SEMISUPNET.BURN_UP_STEP:

        teacher_updated = False

        if False:

            # input both strong and weak supervised data into model
            label_data_q.extend(label_data_k)

            record_dict, _, _, _ = self.model(
                label_data_q, branch="supervised")

            # weight losses
            loss_dict = {}
            for key in record_dict.keys():
                if key[:4] == "loss":
                    loss_dict[key] = record_dict[key] * 1
            losses = sum(loss_dict.values())

        else:
            # if self.iter == self.cfg.SEMISUPNET.BURN_UP_STEP:
            if not teacher_updated:
                # update copy the the whole model
                self._update_teacher_model(keep_rate=0.00)
                teacher_updated = True

            elif (
                self.iter - self.cfg.SEMISUPNET.BURN_UP_STEP
            ) % self.cfg.SEMISUPNET.TEACHER_UPDATE_ITER == 0:
                self._update_teacher_model(
                    keep_rate=self.cfg.SEMISUPNET.EMA_KEEP_RATE)

            record_dict = {}
            #  generate the pseudo-label using teacher model
            # note that we do not convert to eval mode, as 1) there is no gradient computed in
            # teacher model and 2) batch norm layers are not updated as well

            with torch.no_grad():
                    
                '''
                images = self.model_teacher.preprocess_image(unlabel_data_k)
                features = self.model_teacher.backbone(images.tensor)
                
                self.model_teacher.proposal_generator.eval()
                proposals_rpn, _ = self.model_teacher.proposal_generator(images, features, None)

                self.model_teacher.roi_heads.eval()
                proposals_roih_unsup_k, ROI_predictions = self.model_teacher.roi_heads(
                    images,
                    features,
                    proposals_rpn,
                    targets=None,
                    # compute_loss=False,
                    # branch='unsup_data_weak'
                    )

                proposals_rpn_unsup_k = proposals_rpn
                '''

                out = self.model_teacher(unlabel_data_k, branch="unsup_data_weak")

                (
                    _,
                    proposals_rpn_unsup_k,
                    proposals_roih_unsup_k,
                    _,
                ) = out

            #  Pseudo-labeling
            cur_threshold = self.cfg.SEMISUPNET.BBOX_THRESHOLD

            joint_proposal_dict = {}
            joint_proposal_dict["proposals_rpn"] = proposals_rpn_unsup_k
            (
                pseudo_proposals_rpn_unsup_k,
                nun_pseudo_bbox_rpn,
            ) = self.process_pseudo_label(
                proposals_rpn_unsup_k, cur_threshold, "rpn", "thresholding"
            )
            joint_proposal_dict["proposals_pseudo_rpn"] = pseudo_proposals_rpn_unsup_k
            # Pseudo_labeling for ROI head (bbox location/objectness)
            pseudo_proposals_roih_unsup_k, _ = self.process_pseudo_label(
                proposals_roih_unsup_k, cur_threshold, "roih", "thresholding"
            )
            joint_proposal_dict["proposals_pseudo_roih"] = pseudo_proposals_roih_unsup_k

            #  add pseudo-label to unlabeled data

            unlabel_data_q = self.add_label(
                unlabel_data_q, joint_proposal_dict["proposals_pseudo_roih"]
            )
            unlabel_data_k = self.add_label(
                unlabel_data_k, joint_proposal_dict["proposals_pseudo_roih"]
            )

            all_label_data = label_data_q + label_data_k
            all_unlabel_data = unlabel_data_q

            record_all_label_data, _, _, _ = self.model(
                all_label_data, branch="supervised"
            )
            record_dict.update(record_all_label_data)

            # hacky method to change classes
            all_unlabel_data = change_classes(all_unlabel_data)


            record_all_unlabel_data, _, _, _ = self.model(
                all_unlabel_data, branch="supervised"
            )
            new_record_all_unlabel_data = {}
            for key in record_all_unlabel_data.keys():
                new_record_all_unlabel_data[key + "_pseudo"] = record_all_unlabel_data[
                    key
                ]
            record_dict.update(new_record_all_unlabel_data)

            # weight losses
            loss_dict = {}
            for key in record_dict.keys():
                if key[:4] == "loss":
                    if key == "loss_rpn_loc_pseudo" or key == "loss_box_reg_pseudo":
                        # pseudo bbox regression <- 0
                        loss_dict[key] = record_dict[key] * 0
                    elif key[-6:] == "pseudo":  # unsupervised loss
                        loss_dict[key] = (
                            record_dict[key] *
                            self.cfg.SEMISUPNET.UNSUP_LOSS_WEIGHT
                        )
                    else:  # supervised loss
                        loss_dict[key] = record_dict[key] * 1

            losses = sum(loss_dict.values())

        metrics_dict = record_dict
        metrics_dict["data_time"] = data_time
        self._write_metrics(metrics_dict)

        self.optimizer.zero_grad()
        losses.backward()
        self.optimizer.step()

    def _write_metrics(self, metrics_dict: dict):
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }

        # gather metrics among all workers for logging
        # This assumes we do DDP-style training, which is currently the only
        # supported method in detectron2.
        all_metrics_dict = comm.gather(metrics_dict)
        # all_hg_dict = comm.gather(hg_dict)

        if comm.is_main_process():
            if "data_time" in all_metrics_dict[0]:
                # data_time among workers can have high variance. The actual latency
                # caused by data_time is the maximum among workers.
                data_time = np.max([x.pop("data_time")
                                   for x in all_metrics_dict])
                self.storage.put_scalar("data_time", data_time)

            # average the rest metrics
            metrics_dict = {
                k: np.mean([x[k] for x in all_metrics_dict])
                for k in all_metrics_dict[0].keys()
            }

            # append the list
            loss_dict = {}
            for key in metrics_dict.keys():
                if key[:4] == "loss":
                    loss_dict[key] = metrics_dict[key]

            total_losses_reduced = sum(loss for loss in loss_dict.values())

            self.storage.put_scalar("total_loss", total_losses_reduced)
            if len(metrics_dict) > 1:
                self.storage.put_scalars(**metrics_dict)

    @torch.no_grad()
    def _update_teacher_model(self, keep_rate=0.996):
        if comm.get_world_size() > 1:
            student_model_dict = {
                key[7:]: value for key, value in self.model.state_dict().items()
            }
        else:
            student_model_dict = self.model.state_dict()

        new_teacher_dict = OrderedDict()
        for key, value in self.model_teacher.state_dict().items():
            if key in student_model_dict.keys():
                new_teacher_dict[key] = (
                    student_model_dict[key] *
                    (1 - keep_rate) + value * keep_rate
                )
            else:
                raise Exception("{} is not found in student model".format(key))

        self.model_teacher.load_state_dict(new_teacher_dict)

    @torch.no_grad()
    def _copy_main_model(self):
        # initialize all parameters
        if comm.get_world_size() > 1:
            rename_model_dict = {
                key[7:]: value for key, value in self.model.state_dict().items()
            }
            self.model_teacher.load_state_dict(rename_model_dict)
        else:
            self.model_teacher.load_state_dict(self.model.state_dict())

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name)

    def build_hooks(self):
        cfg = self.cfg.clone()
        cfg.defrost()
        cfg.DATALOADER.NUM_WORKERS = 0  # save some memory and time for PreciseBN

        ret = [
            hooks.IterationTimer(),
            hooks.LRScheduler(self.optimizer, self.scheduler),
            hooks.PreciseBN(
                # Run at the same freq as (but before) evaluation.
                cfg.TEST.EVAL_PERIOD,
                self.model,
                # Build a new data loader to not affect training
                self.build_train_loader(cfg),
                cfg.TEST.PRECISE_BN.NUM_ITER,
            )
            if cfg.TEST.PRECISE_BN.ENABLED and get_bn_modules(self.model)
            else None,
        ]

        # Do PreciseBN before checkpointer, because it updates the model and need to
        # be saved by checkpointer.
        # This is not always the best: if checkpointing has a different frequency,
        # some checkpoints may have more precise statistics than others.
        if comm.is_main_process():
            ret.append(
                hooks.PeriodicCheckpointer(
                    self.checkpointer, cfg.SOLVER.CHECKPOINT_PERIOD
                )
            )

        def test_and_save_results_student():
            self._last_eval_results_student = self.test(self.cfg, self.model)
            _last_eval_results_student = {
                k + "_student": self._last_eval_results_student[k]
                for k in self._last_eval_results_student.keys()
            }
            return _last_eval_results_student

        def test_and_save_results_teacher():
            self._last_eval_results_teacher = self.test(
                self.cfg, self.model_teacher)
            return self._last_eval_results_teacher

        ret.append(hooks.EvalHook(cfg.TEST.EVAL_PERIOD,
                   test_and_save_results_student))
        ret.append(hooks.EvalHook(cfg.TEST.EVAL_PERIOD,
                   test_and_save_results_teacher))

        if comm.is_main_process():
            # run writers in the end, so that evaluation metrics are written
            ret.append(hooks.PeriodicWriter(self.build_writers(), period=20))
        return ret


args = default_argument_parser().parse_args()
args.config_file = 'configs/coco_supervision/faster_rcnn_R_50_FPN_sup2_run1.yaml'
cfg = setup(args)
if cfg.SEMISUPNET.Trainer == "ubteacher":
    Trainer = UBTeacherTrainer
elif cfg.SEMISUPNET.Trainer == "baseline":
    Trainer = BaselineTrainer
else:
    raise ValueError("Trainer Name is not found.")

cfg.defrost()
cfg.SOLVER.IMG_PER_BATCH_LABEL = 1
cfg.SOLVER.IMG_PER_BATCH_UNLABEL = 1 
cfg.freeze()
print(cfg.SOLVER.IMG_PER_BATCH_LABEL)
print(cfg.SOLVER.IMG_PER_BATCH_UNLABEL)

trainer = UBTeacherTrainer(cfg)
trainer.train()



[02/01 20:02:30 detectron2]: Rank of current process: 0. World size: 1
[02/01 20:02:30 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
numpy                   1.19.5
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.1
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0+cu111 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla P100-PCIE-16GB (arch=6.0)
Driver version          460.32.03
CUDA_HOME               /usr/local/cuda
Pillow                  7.1.2
torchvision             0.11.1+cu111 @/usr/local/lib/python3.7/dist-packages/torchvision
torchvision arch fl

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
Exception during training:
Traceback (most recent call last):
  File "<ipython-input-33-5fc661fef565>", line 120, in train_loop
    self.run_step_full_semisup()
  File "<ipython-input-33-5fc661fef565>", line 323, in run_step_full_semisup
    all_unlabel_data, branch="supervised"
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/content/drive/MyDrive/PyPSA_Africa_images/unbiased-teac

Instances(num_instances=68, image_height=672, image_width=672, fields=[gt_boxes: Boxes(tensor([[492.8990, 520.3275, 672.0000, 523.6580],
        [542.0302,   0.0000, 542.1142,   0.0000],
        [139.4070,   0.0000, 139.4738,   0.0000],
        [338.6193, 672.0000, 518.1135, 672.0000],
        [395.7135,   0.0000, 396.5756,   0.0000],
        [227.4484,   0.0000, 263.5964,   0.0000],
        [145.3668,   0.0000, 672.0000,   0.0000],
        [659.8271,   0.0000, 659.8577,   0.0000],
        [579.2905,   0.0000, 582.6321,   0.0000],
        [376.9705,   0.0000, 422.2586,   0.0000],
        [160.1147,   0.0000, 162.6877,   0.0000],
        [623.0050, 662.6196, 642.1052, 672.0000],
        [322.4655,   0.0000, 359.4986,   0.0000],
        [184.9034,  25.2306, 427.3770,  60.6626],
        [  0.0000, 550.3253, 672.0000, 666.6003],
        [  0.0000,   0.0000,  11.0409, 672.0000],
        [398.2402, 672.0000, 404.0453, 672.0000],
        [  0.0000, 490.4160, 672.0000, 494.4430],
        [  0.

AssertionError: ignored

In [116]:
from google.colab.patches import cv2_imshow

from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import detection_utils as utils
import matplotlib.pyplot as plt
plt.style.use('bmh')

def show_data(datalist, metadata=MetadataCatalog.get('fake_maxar_val')):

    try:
        datalist = change_classes(datalist)
    except KeyError:
        pass

    for data in datalist:
        img = data["image"].permute(1, 2, 0).cpu().detach().numpy()
        img = utils.convert_image_to_rgb(img, cfg.INPUT.FORMAT)        

        visualizer = Visualizer(img, metadata=metadata, scale=0.6)

        if 'instances' in data:
            target_fields = data["instances"].get_fields()
            labels = [metadata.thing_classes[i] for i in target_fields["gt_classes"]]
            visualizer = visualizer.overlay_instances(
                labels=labels,
                boxes=target_fields.get("gt_boxes", None),
                masks=target_fields.get("gt_masks", None),
                keypoints=target_fields.get("gt_keypoints", None),
            )

        cv2_imshow(visualizer.get_image()[:,:,::-1])



In [117]:
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
import matplotlib.pyplot as plt
plt.style.use('bmh')
import numpy as np

def change_classes(data):
    for i, datum in enumerate(data):
        print(datum['instances'])
        classes = getattr(datum['instances'], 'gt_classes')
        setattr(datum['instances'], 'gt_classes', torch.ones_like(classes, dtype=int))
        data[i] = datum
    
    return data

class MaxarUBTeacher(DefaultTrainer):
    '''
    Unbiased teacher adapted to the transfer of training performance from 
    duke to maxar images
    '''

    def __init__(self, cfg):
        cfg = DefaultTrainer.auto_scale_workers(cfg, comm.get_world_size())

        model = self.build_model(cfg)
        self.model_teacher = self.build_model(cfg)

        data_loader = self.build_train_loader(cfg)
        optimizer = self.build_optimizer(cfg, model)

        if comm.get_world_size() > 1:
            model = DistributedDataParallel(
                model, device_ids=[comm.get_local_rank()], broadcast_buffers=False
            )

        TrainerBase.__init__(self)
        self._trainer = SimpleTrainer(model, data_loader, optimizer)

        ensem_ts_model = EnsembleTSModel(self.model_teacher, model)

        self.scheduler = self.build_lr_scheduler(cfg, optimizer)
        self.checkpointer = DetectionCheckpointer(
            ensem_ts_model,
            cfg.OUTPUT_DIR,
            optimizer=optimizer,
            scheduler=self.scheduler,
        )
        self.start_iter = 0
        self.max_iter = cfg.SOLVER.MAX_ITER
        self.cfg = cfg

        self.register_hooks(self.build_hooks())
        print('setup ub teacher complete!')

    
    def train(self):
        self.train_loop(self.start_iter, self.max_iter)
        if hasattr(self, "_last_eval_results") and comm.is_main_process():
            verify_results(self.cfg, self._last_eval_results)
            return self._last_eval_results


    def train_loop(self, start_iter: int, max_iter: int):
        logger = logging.getLogger(__name__)
        logger.info('Starting from iteration {}'.format(start_iter))

        self.iter = self.start_iter = start_iter
        self.max_iter = max_iter

        with EventStorage(start_iter) as self.storage:
            try:
                self.before_train()

                for self.iter in range(start_iter, max_iter):
                    self.before_step()
                    self.run_step_full_semisup()

                    return


                    self.after_step()
            except Exception:
                logger.exception("Exception during training:")
                raise
            finally:
                self.after_train()


    def remove_label(self, label_data):
        for label_datum in label_data:
            if "instances" in label_datum.keys():
                del label_datum["instances"]
        return label_data


    def run_step_full_semisup(self):
        self._trainer.iter = self.iter
        assert self.model.training, 'self.model was changed to eval mode!'

        start = time.perf_counter()
        data = next(self._trainer._data_loader_iter)
        label_data_q, label_data_k, unlabel_data_q, unlabel_data_k = data
        data_time = time.perf_counter() - start
        # print('unlabel_data_k')
        # show_data(unlabel_data_k)

        # remove unlabeled data labels
        unlabel_data_q = self.remove_label(unlabel_data_q)
        unlabel_data_k = self.remove_label(unlabel_data_k)

        print('unlabel_data_k')
        show_data(unlabel_data_k)
        print('unlabel_data_q')
        show_data(unlabel_data_q)





        print('step done')



    
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = MaxarDatasetMapper(cfg, True)
        # return build_detection_semisup_train_loader(cfg, mapper=None)
        return build_maxar_loader_semisup_two_crops(cfg, mapper)
            



In [118]:
args = default_argument_parser().parse_args()
args.config_file = 'configs/coco_supervision/faster_rcnn_R_50_FPN_sup2_run1.yaml'
cfg = setup(args)
cfg.defrost()
cfg.SEMISUPNET.Trainer = 'ubteacher'
cfg.SOLVER.IMG_PER_BATCH_LABEL = 1
cfg.SOLVER.IMG_PER_BATCH_UNLABEL = 1
cfg.INPUT.FORMAT = 'RGB'
cfg.DATALOADER.ASPECT_RATIO_GROUPING = True
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.INPUT.MIN_SIZE_TRAIN = (256, 512, 640, 672, 704, 736, 768, 800)
cfg.freeze()

trainer = MaxarUBTeacher(cfg)
trainer.train()



[02/13 23:49:17 detectron2]: Rank of current process: 0. World size: 1
[02/13 23:49:17 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 (default, Jan 15 2022, 18:48:18) [GCC 7.5.0]
numpy                   1.19.5
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.1
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0+cu111 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla P100-PCIE-16GB (arch=6.0)
Driver version          460.32.03
CUDA_HOME               /usr/local/cuda
Pillow                  7.1.2
torchvision             0.11.1+cu111 @/usr/local/lib/python3.7/dist-packages/torchvision
torchvision arch fl

Exception during training:
Traceback (most recent call last):
  File "<ipython-input-117-ca1a7333df32>", line 77, in train_loop
    self.run_step_full_semisup()
  File "<ipython-input-117-ca1a7333df32>", line 113, in run_step_full_semisup
    show_data(unlabel_data_k)
  File "<ipython-input-116-f9b5c331b375>", line 32, in show_data
    cv2_imshow(visualizer.get_image()[:,:,::-1])
AttributeError: 'Visualizer' object has no attribute 'get_image'


unlabel_data_k
[02/13 23:50:01 d2.engine.hooks]: Total training time: 0:00:00 (0:00:00 on hooks)
[02/13 23:50:01 d2.utils.events]:  iter: 0    lr: N/A  max_mem: 1150M


AttributeError: ignored

In [73]:
print(cfg.INPUT.MIN_SIZE_TRAIN)
print(cfg.INPUT.MAX_SIZE_TRAIN)
print(cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING)


(640, 672, 704, 736, 768, 800)
1333
choice
